<p style="text-align: center;"><strong><span style="font-size: 24px;">FDML Laser Simulation using Split Step Fourier Method</span></strong></p>

<p style="text-align: center;">Method:  SSF Solution to the Nonlinear Schrödinger Equation</p> 

This simulation models a Fourier Domain Mode Locked (FDML) laser using the Split Step Fourier method. The laser cavity consists of a finite number of building blocks, including Semiconductor Optical Amplifiers (SOAs), a tunable filter, and a long waveguide delay line. It is designed to be self-starting from spontaneous emission. Each block in the cavity is defined by its type, length, material parameters, and arrangement, allowing flexibility in simulating different laser configurations. The goal is to analyze the dynamic behavior of the FDML laser system.

<p style="text-align: center;"><strong><span style="font-size: 20px;">1. Principle</span></strong></p>

<p style="text-align: center;">Example: Propagation of a transform-limited optical pulse through dispersive waveguide media.</p>

<strong><span style="font-size: 20px;">1.1. Define constants and parameters</span></strong>

In [1]:
'''
IMPORT LIBRARIES AND DEPENDENCIES
'''


import numpy as np
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt

# InP SOA Gain Profile (Adopted from SmartPhotonics Design Manual)
from SSF_Dependencies import Gain_Calc, G_Bezier

In [ ]:
'''
DEFINE SPECTRAL BANDWIDTH OF THE INPUT OPTICAL PULSE
'''

# Speed of light in vacuum 
c0 = 3e8                                                # [m/s]

# Wavelength range: Defines the spectral content of the input transform-limited optical pulse
lambda_min = 1530e-9                                    # [m]
lambda_max = 1570e-9                                    # [m]
lambda_0   = 1550e-9                                    # [m]

# Convert wavelength range to frequency range
f_min = c0 / lambda_max
f_max = c0 / lambda_min
f_0   = c0 / lambda_0

# Convert to angular frequency
omega_0 = 2 * np.pi * f_0

In [ ]:
'''
CALCULATE PULSE'S TIME WINDOW, TIME SAMPLE POINTS, FREQUENCY SAMPLE POINTS
'''

# Calculate the spectral bandwidth
delta_f = f_max - f_min                                 # [Hz]

# Convert the spectral bandwidth to angular frequency 
delta_omega = 2 * np.pi * delta_f                       # [rad/s]

# Estimate the pulse duration in seconds (for a Gaussian pulse with a Time Bandwidth Product of ~ 0.44)
delta_t = 0.44 / delta_f                                # [s]

# Total time window of the optical pulse: Should be several times the pulse duration (Retarded Time)
T = 20*delta_t  #!!! If you increase T, you must increase N approproately

'''
Number of sample points for the time-window. (i.e. sample points for the retarded time)
N must be chosen accordingly to balance resolution and computational cost.
'''
N = 2048  # Sampling points (Time-Domain)

# Time step
dt = T / N                                              # [s]

# Time grid: Contains all the sampled timepoints
t = np.linspace(-T/2, T/2, N)                           # [s]

# Sampling frequency (Spectral-Domain)
fs = N / T
print("FFT Sampling Frequency is: {:.3e} Hz".format(fs))

# Frequency bin: Calculate the frequency bins required for the Fast-Fourier Transforms
delta_f = fs/N                                          # [Hz]
delta_omega = delta_f * 2 * np.pi                       # [rad/s]

# Define the frequency grid (angular frequency)
f = np.fft.fftfreq(N, d=dt)  # Frequency                # [Hz]
omega = 2 * np.pi * f        # Angular frequency        # [rad/s]

In [ ]:
'''
PARAMETERS: PROPAGATION REGION
'''

dz = 15.5                                               # [um] Spatial step size (Segment's Physical Length)
num_steps = 645                                         # Number of spatial points/segments
n_g = 3.665                                             # Group Index of Refraction for 1550nm
v_g = c0/n_g                                            # [m/s] Group velocity

In [ ]:
'''
PARAMETERS: INDIVIDUAL SEGMENT
'''

gamma = 36.85e-6                                        # [1/W/um] Nonlinear coefficient  
# Source: S. Saeidi et al., "Demonstration of Optical Nonlinearity in InGaAsP/InP Passive Waveguides"

# Optical Power Loss
loss_m = -340                                           # [dB] Waveguide loss per meter 
loss = loss_m * dz * 1e-6                               # [dB] Waveguide loss per segment 

# Dispersion
D = 500e-6                                              # Fiber dispersion [s/m/m], originally [ps/nm/km]

In [ ]:
'''
DEFINE FUNCTIONS TO: APPLY DISPERSION, APPLY GAIN/LOSS, APPLY NONLINEARITY, APPLY ABSOLUTE TIME-DEPENDENT EFFECTS
'''

# Applies Group Velocity Dispersion
def beta(omega):
    return -1j * np.pi * c0 * D * (omega/omega_0)**2 # Group Velocity Dispersion [1/m]

# Applies Gain/Loss: (loss: passive waveguide, gain: SOA) 
def gain_loss_function(gain, loss, omega):
    # Divisor is 20 to covert Power Loss to Envelope Loss     
    return 10**((gain+loss)/20)   

# Applies Nonlinearity
def gamma_nonlinearity(A):
    return np.exp(1j * gamma * np.abs(A)**2 * dz)

'''
ABSOLUTE TIME: refers to the time that has ellapsed by the pulse propagating through a certain number of segments.
Is different from the RETARDED TIME, that refers to the time-domain representation of the optical pulse in a single
segment.
'''

# Define Absolute-Time Time-Dependent Effects (i.e. Tunable Filter)
def time_dependent_effect(A, t_abs):
    # Example time-dependent effect: apply a phase shift depending on absolute time
    phase_shift = np.exp(01j * 0.01 * t_abs)
    return A * phase_shift

In [2]:
'''
A FUNCTION THAT PLOTS THE INITIAL OPTICAL PULSE vs THE PROPAGATED OPTICAL PULSE AFTER PROPAGATING THROUGH N=num_steps SEGMENTS OF LENGTH L=dz. 
IT ALSO PLOTS THE CHIRP OF THE INITIAL OPTICAL PULSE (no-chirp here) VS THE PROPAGATED OPTICAL PULSE. FINALLY, IT PLOTS THE POWER SPECTRUM.
'''

# Define Output FFT Plots
def out_plot(omega, A0, A_hat):
    A0_FFT = np.fft.fftshift(fft(A0))
    AF_FFT = np.fft.fftshift(A_hat)
    omega_centered = np.fft.fftshift(omega)
    
    # Magnitude Squared
    A0_FFT_M = delta_omega * (np.abs(A0_FFT)**2)/((2*np.pi*fs)*N)
    AF_FFT_M = delta_omega * (np.abs(AF_FFT)**2)/((2*np.pi*fs)*N)
    
    # Phase
    A0_FFT_P = np.angle(A0_FFT)*180/np.pi
    AF_FFT_P = np.angle(AF_FFT)*180/np.pi
    
    
    # Plot
    fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1)
    
    # Plot the Magnitude
    ax1.plot(omega_centered*1e-12/(2*np.pi), A0_FFT_M * 1e3, omega_centered*1e-12/(2*np.pi), AF_FFT_M * 1e3)
    ax1.legend(['Initial','Propagated'])
    ax1.set_title('Power Spectrum of the Optical Envelope')
    #ax1.set_xlabel(r'Frequency $\nu$ [THz]')
    ax1.set_xlim([-30, 30])
    ax1.set_ylabel(r'Magnitude [$\mu W$]')
    #plt.plot()
    
    
    # Plot the Phase
    ax2.plot(omega_centered*1e-12/(2*np.pi), A0_FFT_P, omega_centered*1e-12/(2*np.pi), AF_FFT_P)
    ax2.legend(['Initial','Propagated'])
    ax2.set_title('Phase Angle')
    ax2.set_xlabel(r'Frequency $\nu$ [THz]')
    ax2.set_xlim([-30, 30])
    ax2.set_ylabel(r'Phase [Deg]')
    plt.plot()

In [ ]:
'''
DEFINE THE SSF ALGORITHM
'''

def ssfm(A0, dz, num_steps, beta, v_g, gamma=None, gain_loss_function=None, time_dependent_segments=None, gain_segments=None):
    
    # A0 is the initial optical pulse
    A = A0

    # Absolute time starting point
    t_abs = 0  

    for n in range(num_steps):

        '''
        The absolute time is approximated using the group velocity and the segment length!
        '''
        # Update absolute time
        t_abs += dz / v_g
        
        # Fourier transform of the pulse
        A_hat = fft(A)

        # APPLY DISPERSION: Linear step with dispersion
        A_hat *= np.exp(beta(omega) * dz * 1e-6)
        
        '''
        APPLY GAIN/LOSS IF PROVIDED
        '''
        # Apply loss only if segment is passive waveguide
        if gain_loss_function is not None and n not in gain_segments:
                A_hat += (gain_loss_function(0, loss, omega) - 1)*A_hat 
        
        if gain_segments and n in gain_segments:
            #if n == gain_segments[0]:
            PSD = (np.abs(A_hat)**2)/((2*np.pi*fs)*N)
            PS = PSD * delta_omega
            '''
            Gain_Calc takes as input the power of a certain frequency/wavelength
            of light at the entrance of a 500um SOA gain block and returns the 
            G_dB_um (Power Gain in dB per micron). This gain can later be used
            for amplification without the need to recalculate the Power Spectrum
            (PS) at every gain segment. The PS is only needed at the entrance to
            the gain block.
            '''
            
            # Power Gain per Segment 
            dB_Gain = dz * Gain_Calc(PS, dz, omega + omega_0)[1]
            dB_Gain = np.transpose(dB_Gain) #+ 1e-323
        
            print("Gain per segment for 1550nm in dB: {}".format(dB_Gain[0]))
            #print(gain_loss_function(dB_Gain, loss, omega)[:,0])
            #print("Power Gain in dB per Segment is: {}".format(dB_Gain))
            #_loss_ = loss * np.ones((len(omega)))
            A_hat += (gain_loss_function(dB_Gain, loss, omega)[:,0] - 1)*A_hat 
            #else:
            #    A_hat += (gain_loss_function(dB_Gain, loss, omega)[:,0] - 1)*A_hat 
        
        # Inverse Fourier transform to time domain
        A = ifft(A_hat)
        
        # Apply Kerr Nonlinearity
        if gamma is not None:
            A *= gamma(A)
        
        # Apply time-dependent effect if in specified segments
        if time_dependent_segments and n in time_dependent_segments:
            A = time_dependent_effect(A, t_abs)
            
        if n == num_steps-1:
            # Plot the FFTs
            out_plot(omega, A0, A_hat)
            

    return A, t_abs, PS